In [1]:
import pandas as pd

df = pd.read_csv("fr-2017-pssd-compendium-20180320-utf8.csv")
df.head()

,Secteur,Nom de famille,Prénom,Traitement versé,Avantages imposables,Employeur,Poste,Année civile
0,Autres employeurs du secteur public,Coleridge,Peter,"$250,000.00","$25,749.50",Big Brothers Big Sisters of Canada,National President and Chief Executive Officer,2017
1,Autres employeurs du secteur public,Chater,W. Matthew,"$100,877.00","$2,778.27",Big Brothers Big Sisters of Canada,"National Vice-President, Service and Organizat...",2017
2,Autres employeurs du secteur public,Mcintyre,Mary Ellen,"$113,847.58","$19,015.00",Centre For Spanish Speaking Peoples,Legal Director,2017
3,Autres employeurs du secteur public,Marlin,Susan,"$248,068.75",$258.48,Clinical Trials Ontario,President and Chief Executive Officer,2017
4,Autres employeurs du secteur public,Frampton,Peter,"$165,033.85","$10,788.30",Learning Enrichment Foundation,Executive Director,2017


In [2]:
df["salary"] = df['Traitement versé'
                           ].replace( '[\$,]','', regex=True
                           ).astype(float
                           ) + df['Avantages imposables'
                                 ].replace( '[\$,]','', regex=True
                                 ).astype(float)

In [3]:
# Standardize strings to ascii form
from unidecode import unidecode 
df['name'] = df['Prénom'].apply(lambda n : unidecode(n))

In [4]:
genders =  pd.read_csv("us-likelihood-of-gender-by-name-in-2014.csv")
genders.head()

,sex,name,gender.prob
0,F,Elaine,1.0
1,F,Cathy,1.0
2,F,Heidi,1.0
3,F,Vicki,1.0
4,F,Melinda,1.0


In [5]:
df = df.merge(genders, on="name")
df.sex = df.sex.map({'M': 1, "F":0}).astype('int8')
df = df.drop(["Nom de famille", "Traitement versé", 
              "Avantages imposables", "Année civile",
              "Prénom"], 1)

In [6]:
import statsmodels.api as sm
import numpy as np

X = sm.add_constant(df.sex)
sm.OLS(np.log(df.salary), X).fit().summary()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 salary   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1105.
Date:                Tue, 10 Jul 2018   Prob (F-statistic):          5.80e-241
Time:                        20:32:15   Log-Likelihood:                 11362.
No. Observations:              101732   AIC:                        -2.272e+04
Df Residuals:                  101730   BIC:                        -2.270e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.7026      0.001   1.13e+04      0.000      11.701      11.705
sex            0.0456      0.001     33.239      0.000       0.043       0.048
==============================================================================
Omnibus:                    42964.745   Durbin-Watson:                   1.648
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           219409.558
Skew:                           2.018   Prob(JB):                         0.00
Kurtosis:                       8.956   Cond. No.                         2.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
X = sm.add_constant(df.sex
      ).join(
        pd.get_dummies(df.Employeur, 
                       sparse=True, # This is necessary due to data size!
                       drop_first=True)
    )
gc.collect()

sm.OLS(np.log(df.salary), X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 salary   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     18.35
Date:                Tue, 10 Jul 2018   Prob (F-statistic):               0.00
Time:                        20:35:15   Log-Likelihood:                 24355.
No. Observations:              101732   AIC:                        -4.538e+04
Df Residuals:                  100067   BIC:                        -2.951e+04
Df Model:                        1664                                         
Covariance Type:            nonrobust                                         
==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             11.4722      0.192     59.739      0.000      11.096      11.849
sex                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                0.0408      0.001     30.196      0.000       0.038       0.043
Ableliving Services Inc                                                                                                                                                                                                                                                     

# Meddling around with gendered names

In [13]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
